In [1]:
ROWID = ['f_0']
DATE = ['f_1']
CATEGORIES = [ f'f_{i}' for i in range(2,33) ]
BINARY = [ f'f_{i}' for i in range(33,42) ]
NUMERICAL = [ f'f_{i}' for i in range(42,80) ]
IS_CLICKED = ['is_clicked']
IS_INSTALLED =['is_installed']

In [1]:
import torch
import torch.nn as nn

In [2]:
model = nn.Sequential(
    nn.Linear(78, 200),
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.ReLU(),
    nn.Linear(100, 40),
    nn.ReLU(),
    nn.Linear(40,1),
    nn.Sigmoid()
)

In [3]:
device = 'cuda'

In [4]:
model.to(device)

Sequential(
  (0): Linear(in_features=78, out_features=200, bias=True)
  (1): ReLU()
  (2): Linear(in_features=200, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=40, bias=True)
  (5): ReLU()
  (6): Linear(in_features=40, out_features=1, bias=True)
  (7): Sigmoid()
)

In [5]:
x = torch.rand((10,78)).to(device)

In [6]:
x.shape

torch.Size([10, 78])

In [7]:
out = model(x)

In [8]:
out.shape

torch.Size([10, 1])

In [9]:
out

tensor([[0.4743],
        [0.4691],
        [0.4736],
        [0.4726],
        [0.4707],
        [0.4697],
        [0.4744],
        [0.4731],
        [0.4727],
        [0.4702]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [10]:
from utils import TrainSplit,TestResults

In [11]:
X_train, X_test, y_train, y_test = TrainSplit().get_split()

Loading the data
Categorial Feature Imputed


NUM IMPUTE: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 48.21it/s]


Spliting the Data based on time
X_train:(3387880, 80), X_test:(97972, 80) , y_train:(3387880, 2) , y_test:(97972, 2)


In [12]:
cat_feat = TrainSplit.CATEGORIES + TrainSplit.BINARY
num_feat = TrainSplit.NUMERICAL
all_feat = cat_feat + num_feat

In [13]:
type(X_train)

pandas.core.frame.DataFrame

In [14]:
type(y_train)

pandas.core.frame.DataFrame

In [15]:
from torch.utils.data import Dataset

class Features(Dataset):
    
    def __init__(self,config):
        self.config = config
        self.dataset,self.label = self.train_test_split()
        print("Train Test Split")
    
    def train_test_split(self):
        if self.config["split"] == 'train':
            return X_train , y_train[TrainSplit.IS_INSTALLED[0]]
        elif self.config['split'] == 'val':
            return X_test , y_test[TrainSplit.IS_INSTALLED[0]]
        else:
            raise Exception(f"{self.config['split']} is vot a valid split only type")
    
    def __getitem__(self,idx):
            return self.dataset.iloc[idx,:][all_feat].values,self.label.iloc[idx]
        
    def __len__(self):
        return len(self.dataset)

In [16]:
train_dataset = Features({'split':'train'})

Train Test Split


In [17]:
len(train_dataset)/131072

25.84747314453125

In [18]:
import gc
gc.collect()

0

In [19]:
val_dataset = Features({'split':'val'})

Train Test Split


In [20]:
len(val_dataset)

97972

In [21]:
len(val_dataset[0][0])

78

In [22]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

In [24]:
len(train_dataloader)

211743

In [25]:
# 131072

In [26]:
for n in range(2):
    count = 0
    cum_loss = 0
    for X,y in train_dataloader:
        print(X)
        X = torch.tensor(X,dtype=torch.float32,device=device)
        print(X)
        y = torch.tensor(y,dtype=torch.long,device=device)
        print(y)
        y_pred = model(X)
        print(y_pred)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        cum_loss += loss
        print(f"Done {count/len(train_dataloader)}% of epoch {n+1} the loss is {cum_loss}")
        count+=1
    print(f"Epoch {n},Loss {loss}")

tensor([[11077.,  7152., 18575.,  ...,     0.,     0.,     0.],
        [20095.,   563., 31686.,  ...,     0.,     0.,     0.],
        [ 5156., 22294., 18971.,  ...,     0.,     0.,     0.],
        ...,
        [12577., 22294.,  2059.,  ...,     0.,     0.,     0.],
        [25976., 22294.,  7696.,  ...,     0.,     0.,     0.],
        [31824., 22294., 10633.,  ...,     0.,     0.,     0.]],
       dtype=torch.float64)
tensor([[11077.,  7152., 18575.,  ...,     0.,     0.,     0.],
        [20095.,   563., 31686.,  ...,     0.,     0.,     0.],
        [ 5156., 22294., 18971.,  ...,     0.,     0.,     0.],
        ...,
        [12577., 22294.,  2059.,  ...,     0.,     0.,     0.],
        [25976., 22294.,  7696.,  ...,     0.,     0.,     0.],
        [31824., 22294., 10633.,  ...,     0.,     0.,     0.]],
       device='cuda:0')
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], device='cuda:0')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        

/tmp/ipykernel_28630/1837055144.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X,dtype=torch.float32,device=device)
/tmp/ipykernel_28630/1837055144.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,dtype=torch.long,device=device)
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [13,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_update

RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`

In [ ]:
from torcheval.metrics import BinaryF1Score,BinaryPrecision,BinaryRecall
val_dataloader = DataLoader(val_dataset, batch_size=1024, shuffle=True)